# Model - info

In [ ]:
import pandas as pd 
from pathlib import Path
import networkx as nx
import igraph as ig
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from scipy.spatial import distance
import seaborn as sns
import time
from tqdm.auto import tqdm
import random 
import os
from itertools import chain, combinations
import itertools
import scipy

import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
plt.rcParams.update({'font.size': 22})
sns.set(style="ticks", context="talk")
plt.style.use("dark_background")
pd.options.plotting.backend = 'plotly'
pio.templates.default = 'plotly_dark+presentation'

import numpy as np
from scipy.optimize import minimize

#pd.options.mode.chained_assignment = None 

tqdm.pandas()

import matplotlib.dates as dates
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
from scipy import optimize
import numpy.polynomial.polynomial as npoly

def form(x,pos):
    if x<1e3:
        return '%1.3f' % (x)
    elif x<1e6:
        return '%1.1fK' % (x * 1e-3)
    else:
        return '%1.1fM' % (x * 1e-6)
formatter = FuncFormatter(form)

import warnings
warnings.filterwarnings("ignore")

def read_parquet(name, **args):
    path = name
    print(f'Reading {name!r}')
    tic = time.time()
    df = pd.read_parquet(path, engine='fastparquet', **args)
    before = len(df)
    # df.drop_duplicates(inplace=True)
    toc = time.time()
    after = len(df)
        
    print(f'Read {len(df):,} rows from {path.stem!r} in {toc-tic:.2f} sec. {before-after:,} duplicates.')
    return df

In [ ]:
def my_weight(G, u, v, weight="weight"):
    w = 0
    for nbr in set(G[u]) & set(G[v]):
        w += (G[u][nbr].get(weight, 1) + G[v][nbr].get(weight, 1))/2
    return w
def make_institution_graph(works_authors_rows):
    
    institution_id_set = set(works_authors_rows.institution_id)
                                  
    bip_g = nx.from_pandas_edgelist(
        works_authors_rows,
        source='work_id', target='institution_id', edge_attr ='weight'
    )

    inst_graph = nx.bipartite.generic_weighted_projected_graph(bip_g,nodes=institution_id_set,weight_function=my_weight)    
    #inst_graph = nx.bipartite.weighted_projected_graph(bip_g,nodes=institution_id_set) 

    return inst_graph


In [ ]:
basepath = Path('./Tables_final') 
my_path_ = Path('./Model_info')
if not os.path.exists(my_path_):
    os.makedirs(my_path_)

## Perc. works resp. team size

In [ ]:
works = read_parquet(basepath / 'works')

#TW training window
months_list = list(set(works.reset_index().drop_duplicates('publication_date_1').publication_date_1))
months_list.sort()
months_list = [i.strftime('%Y-%m-%d') for i in months_list]
start_index = 0
end_index = 120 #180
start_month = months_list[start_index]#.strftime('%Y-%m-%d')
end_month = months_list[end_index-1]#.strftime('%Y-%m-%d')
print(start_month,end_month)
works = works[works.num_authors > 1]
works_TW = works.loc[start_month:end_month]
works_AW = works.loc[months_list[end_index]:]

df_perc_works = ((works.groupby('num_authors').work_id.count().to_frame().cumsum()/len(works))*100).rename(columns={'work_id':'perc_works'})
display(df_perc_works)
df_perc_works = ((works_AW.groupby('num_authors').work_id.count().to_frame().cumsum()/len(works_AW))*100).rename(columns={'work_id':'perc_works'})
display(df_perc_works)

## Edges - 2 authors

In [ ]:
path_2authors = Path('./Model_2authors') 
if not os.path.exists(path_2authors):
    os.makedirs(path_2authors)

### Tables

In [ ]:
works = read_parquet(basepath / 'works')
works_authors_aff = read_parquet(basepath / 'works_authors_aff')

works = works.loc['2000-01-01':'2023-12-01'] 
works = works[works.num_authors>1]

works_2authors = set(works[works.num_authors ==2].work_id)
print(f'{len(works_2authors)} ({(len(works_2authors)/len(works))*100:.2f}%) works 2 authors')

works = works[works.work_id.isin(works_2authors)]
works_authors_aff = works_authors_aff[works_authors_aff.work_id.isin(works_2authors)]

my_file = "dfs_2authors.pickle"
pickle.dump([works_2authors,works,works_authors_aff], open(os.path.join(path_2authors, my_file), 'wb')) 

In [ ]:
#INITIALIZATION #preferential attachment  #TW: 2000-2009
my_file = "dfs_2authors.pickle"
with open(os.path.join(path_2authors, my_file),"rb") as fp:
    [works_2authors,works,works_authors_aff] = pickle.load(fp)  
works = works.loc['2000-01-01':'2023-12-01'] 
works_authors_aff = works_authors_aff.loc['2000-01-01':'2023-12-01']

N_dict = works_authors_aff.groupby('publication_date_1').work_id.nunique().to_frame().to_dict()['work_id']
months_list = list(N_dict.keys())
months_list.sort()
months_list = [i.strftime('%Y-%m-%d') for i in months_list]

start_index = 0
end_index = 120 #180
start_month = months_list[start_index]#.strftime('%Y-%m-%d')
end_month = months_list[end_index-1]#.strftime('%Y-%m-%d')

df_TW = works_authors_aff.loc[start_month:end_month] #df_TW = works_authors_aff.loc[months_list[:120]]
inst_set = set(df_TW.institution_id)
I = len(inst_set)
print(f'TW from {start_month} to {end_month} : {I} institutions')

In [ ]:
#initial strenghts    #consider only institutions in sample in TW
#count number (unique) institutions per paper
df_TW = df_TW.drop_duplicates(['work_id','institution_id'])
df_TW['num_affs'] = df_TW.groupby('work_id')['institution_id'].transform('size')
print(f'{min(df_TW.num_affs)}-{max(df_TW.num_affs)} min-max number (unique) affiliations per work')
df_TW['weight'] = 2 / ( df_TW['num_affs']*(df_TW['num_affs']-1) ) 
df_TW.loc[df_TW.num_affs==1,'weight'] = 1 #one affiliation
df_TW_noloops = df_TW[df_TW.num_affs>1]
df_TW_loops = df_TW[df_TW.num_affs==1]
df_TW_loops['institution_id2'] = df_TW_loops['institution_id']
df_TW_loops['weight'] = df_TW_loops[['institution_id','institution_id2','weight']].groupby(['institution_id']).weight.transform('sum')
df_TW_loops = df_TW_loops.drop_duplicates('institution_id')
I_graph = make_institution_graph(df_TW_noloops)
I_graph.add_weighted_edges_from([tuple(r) for r in df_TW_loops[['institution_id','institution_id2','weight']].to_numpy()])
df_ = pd.DataFrame.from_dict(dict(I_graph.degree(weight='weight')),orient='index').reset_index().rename(columns={'index':'institution_id',0:'strength'})
df_['institution_id'] = df_['institution_id'].astype(int)
df = df_.sort_values(by='strength',ascending=False)
inst_set = set(df.institution_id)
I = len(inst_set)
print(f'TW from {start_month} to {end_month} : {I} institutions')

my_file = "df_strengths0.csv"     
df.to_csv(os.path.join(path_2authors, my_file),index=False)

my_file = "inst_set.pickle"
pickle.dump(inst_set, open(os.path.join(path_2authors, my_file), 'wb'))

In [ ]:
I_dist = read_parquet(basepath / 'I_dist_threshold')
I_dist = I_dist[['source','target','dist']].reset_index(drop=True)
I_dist = I_dist.query('source.isin(@inst_set) & target.isin(@inst_set)').reset_index()
my_file = "I_dist_model.csv" 
I_dist.to_csv(os.path.join(path_2authors, my_file),index=False)

### Data plots

In [ ]:
my_file = "dfs_2authors.pickle"
with open(os.path.join(path_2authors, my_file),"rb") as fp:
    [works_2authors,works,works_authors_aff] = pickle.load(fp)  

N_dict = works_authors_aff.groupby('publication_date_1').work_id.nunique().to_frame().to_dict()['work_id']

months_list = list(N_dict.keys())
months_list.sort()
months_list = [i.strftime('%Y-%m-%d') for i in months_list]

In [ ]:
def plot_fit_rolling_breakpoints_2(df,x_column,x_column2,x_label,title,window_size,num_breakpoints):

    fig, ax = plt.subplots(figsize=(15, 5))
    x_dates = list(df['publication_date_1'])
    y_data = df[x_column].rolling(window=window_size).mean()[window_size-1:]
    y_data2 = df[x_column2].rolling(window=window_size).mean()[window_size-1:]
    x_data = x_dates[window_size-1:]
    ax.set_ylim([0, 1])
    
    ax.plot(x_data, y_data, "o-", color='orange', markersize=3,label='intra-institution')
    ax.plot(x_data, y_data2, "o-", color='green', markersize=3,label='inter-institution')

    plt.grid(True, linewidth=0.5)
    ax.yaxis.set_major_formatter(formatter)

    ax.set_xlabel(x_label,size=20)
    #ax.set_title(title,size=30)

    #ax.xaxis.set_major_locator(mdates.MonthLocator()) # Make ticks on occurrences of each month
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %y')) # Get only the month to show in the x-axis

    ax.axvline(pd.Timestamp(2020, 3, 1),color='r') #ax.axvline(x_dates[84],color='r')

    #save for all possible combination of breakpoints the correspondent error 
    def f(breakpoints, x, y, fcache): 
        breakpoints = tuple(map(int, sorted(breakpoints)))
        if breakpoints not in fcache:
            total_error = 0
            for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x, y):
                total_error += ((f(xi) - yi)**2).sum()
            fcache[breakpoints] = total_error 
        # print('{} --> {}'.format(breakpoints, fcache[breakpoints]))
        return fcache[breakpoints]

    def find_best_piecewise_polynomial(breakpoints, x, y):
        breakpoints = tuple(map(int, sorted(breakpoints)))
        xs = np.split(x, breakpoints)
        ys = np.split(y, breakpoints)
        result = []
        for xi, yi in zip(xs, ys):
            if len(xi) < 2: continue
            coefs = npoly.polyfit(xi, yi, 1)
            f = npoly.Polynomial(coefs)
            result.append([f, xi, yi])
        return result
    
    x_num = dates.date2num(x_data)
    breakpoints = optimize.brute(f, [slice(1, len(x_data), 1)]*num_breakpoints, args=(x_num, y_data, {}), finish=None)
    if num_breakpoints==1:
        breakpoints = [breakpoints] 
    for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x_num, y_data):
        xi_min = x_data[np.where(x_num == xi.min())[0][0]]
        xi_max = x_data[np.where(x_num == xi.max())[0][0]]
        x_interval = np.array([xi_min, xi_max]) 
        #print('y = {:35s}, if x in [{}, {}]'.format(str(f), *x_interval))
        x_interval = np.array([xi.min(), xi.max()])
        #ax.plot(x_interval, f(x_interval), 'yo-')
        coef = f.convert().coef
        b = coef[0]
        a = coef[1]
        if b>0:
            ll = 'y = {:.2f} x + {:.0f}'.format(a,b)
        else: #b<0
            ll = 'y = {:.2f} x - {:.0f}'.format(a,abs(b))    
        ax.plot(x_interval, f(x_interval), 'yo-',linewidth=2, markersize=7)       
    #save for all possible combination of breakpoints the correspondent error
    
    def f(breakpoints, x, y, fcache): 
        breakpoints = tuple(map(int, sorted(breakpoints)))
        if breakpoints not in fcache:
            total_error = 0
            for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x, y):
                total_error += ((f(xi) - yi)**2).sum()
            fcache[breakpoints] = total_error 
        # print('{} --> {}'.format(breakpoints, fcache[breakpoints]))
        return fcache[breakpoints]

    def find_best_piecewise_polynomial(breakpoints, x, y):
        breakpoints = tuple(map(int, sorted(breakpoints)))
        xs = np.split(x, breakpoints)
        ys = np.split(y, breakpoints)
        result = []
        for xi, yi in zip(xs, ys):
            if len(xi) < 2: continue
            coefs = npoly.polyfit(xi, yi, 1)
            f = npoly.Polynomial(coefs)
            result.append([f, xi, yi])
        return result
    
    x_num = dates.date2num(x_data)
    breakpoints = optimize.brute(f, [slice(1, len(x_data), 1)]*num_breakpoints, args=(x_num, y_data2, {}), finish=None)
    if num_breakpoints==1:
        breakpoints = [breakpoints] 
    for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x_num, y_data2):
        xi_min = x_data[np.where(x_num == xi.min())[0][0]]
        xi_max = x_data[np.where(x_num == xi.max())[0][0]]
        x_interval = np.array([xi_min, xi_max]) 
        #print('y = {:35s}, if x in [{}, {}]'.format(str(f), *x_interval))
        x_interval = np.array([xi.min(), xi.max()])
        #ax.plot(x_interval, f(x_interval), 'yo-')
        coef = f.convert().coef
        b = coef[0]
        a = coef[1]
        if b>0:
            ll = 'y = {:.2f} x + {:.0f}'.format(a,b)
        else: #b<0
            ll = 'y = {:.2f} x - {:.0f}'.format(a,abs(b))    
        ax.plot(x_interval, f(x_interval), 'yo-',linewidth=2, markersize=7) 
       
    ax.legend()        
    ax.set_title(title,size=30)

In [ ]:
import matplotlib.dates as dates
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
from scipy import optimize
import numpy.polynomial.polynomial as npoly

def form(x,pos):
    if x<1e3:
        return '%1.3f' % (x)
    elif x<1e6:
        return '%1.1fK' % (x * 1e-3)
    else:
        return '%1.1fM' % (x * 1e-6)
formatter = FuncFormatter(form)

def plot_fit_rolling_breakpoints(df,x_column,x_label,title,window_size,num_breakpoints,ff):
    
    #save for all possible combination of breakpoints the correspondent error 
    def f(breakpoints, x, y, fcache): 
        breakpoints = tuple(map(int, sorted(breakpoints)))
        if breakpoints not in fcache:
            total_error = 0
            for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x, y):
                total_error += ((f(xi) - yi)**2).sum()
            fcache[breakpoints] = total_error 
        # print('{} --> {}'.format(breakpoints, fcache[breakpoints]))
        return fcache[breakpoints]

    def find_best_piecewise_polynomial(breakpoints, x, y):
        breakpoints = tuple(map(int, sorted(breakpoints)))
        xs = np.split(x, breakpoints)
        ys = np.split(y, breakpoints)
        result = []
        for xi, yi in zip(xs, ys):
            if len(xi) < 2: continue
            coefs = npoly.polyfit(xi, yi, 1)
            f = npoly.Polynomial(coefs)
            result.append([f, xi, yi])
        return result

    fig, ax = plt.subplots(figsize=(15, 5))
    x_dates = list(df['publication_date_1'])
    y_data = df[x_column].rolling(window=window_size).mean()[window_size-1:]
    x_data = x_dates[window_size-1:]
    
    ax.plot(x_data, y_data, "o-", color='orange', markersize=3)

    plt.grid(True, linewidth=0.5)
    ax.yaxis.set_major_formatter(formatter)

    ax.set_xlabel(x_label,size=20)
    ax.set_title(title,size=30)

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %y')) # Get only the month to show in the x-axis

    ax.axvline(pd.Timestamp(2020, 3, 1),color='r') #ax.axvline(x_dates[84],color='r')

    x_num = dates.date2num(x_data)
    breakpoints = optimize.brute(f, [slice(1, len(x_data), 1)]*num_breakpoints, args=(x_num, y_data, {}), finish=None)
    if num_breakpoints==1:
        breakpoints = [breakpoints] 

    for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x_num, y_data):
        xi_min = x_data[np.where(x_num == xi.min())[0][0]]
        xi_max = x_data[np.where(x_num == xi.max())[0][0]] 
        x_interval = np.array([xi.min(), xi.max()])
        coef = f.convert().coef
        b = coef[0]
        a = coef[1]
        if ff==1:
            if b>0:
                ll = 'y = {:.2f} x + {:.2f}'.format(a,b)
            else: #b<0
                ll = 'y = {:.2f} x - {:.2f}'.format(a,abs(b))    
        else:
            if b>0:
                ll = 'y = {:.5f} x + {:.5f}'.format(a,b)
            else: #b<0
                ll = 'y = {:.5f} x - {:.5f}'.format(a,abs(b))  
        #x_1 = x_dates[window_size-1:][np.where(x_num==x_interval[0])[0][0]]
        #x_3 = x_dates[window_size-1:][np.where(x_num==x_interval[1])[0][0]]
        ax.plot(x_interval, f(x_interval), 'o-',color='yellow',label=ll, markersize=6)
        
    ax.legend()
    ax.set_title(title,size=30)
    #return x_num 
    
x_label='Month'
window_size = 6
num_breakpoints = 1

In [ ]:
def df_data1_(work_authors_edges_df):
    #work_authors_edges_df['intra'] = 0
    #work_authors_edges_df.loc[work_authors_edges_df.source_inst == work_authors_edges_df.target_inst,'intra'] = 1
    df1 = work_authors_edges_df.groupby('publication_date_1').intra.count().reset_index().rename(columns={'intra':'total'})
    df2 = work_authors_edges_df.groupby('publication_date_1').intra.sum().to_frame().reset_index()
    intra_inter_df = df1.merge(df2,on='publication_date_1')
    intra_inter_df['inter'] = intra_inter_df['total'] - intra_inter_df['intra']
    intra_inter_df['frac_intra'] = intra_inter_df['intra'] / intra_inter_df['total']
    intra_inter_df['frac_inter'] = intra_inter_df['inter'] / intra_inter_df['total']
    df_data1 = intra_inter_df
    return df_data1

def work_edges_dist_mean_monthly_(works_outside,works_set,path):
    my_file = "work_edges_dist_mean.csv"    
    work_edges_dist_mean = pd.read_csv(os.path.join(path, my_file))
    work_edges_dist_mean = work_edges_dist_mean[work_edges_dist_mean.work_id.isin(works_set)]
    work_edges_dist_mean = work_edges_dist_mean[~work_edges_dist_mean.work_id.isin(works_outside)]
    work_edges_dist_mean['publication_date_1'] = pd.to_datetime(work_edges_dist_mean['publication_date_1'])
    work_edges_dist_mean_monthly = work_edges_dist_mean.groupby('publication_date_1').dist.mean().to_frame().reset_index()
    return work_edges_dist_mean_monthly

In [ ]:
work_authors_edges_df_dist = read_parquet(Path('./TeamDistance') / 'work_authors_edges_df_dist')
work_authors_edges_df_dist = works[['work_id']].reset_index().merge(work_authors_edges_df_dist,on='work_id')
work_authors_edges_df_dist = work_authors_edges_df_dist[work_authors_edges_df_dist.work_id.isin(works_2authors)]
work_edges_dist_mean = work_authors_edges_df_dist.groupby('work_id').dist.mean().to_frame().reset_index()
work_edges_dist_mean = work_edges_dist_mean.merge(works[['work_id']].reset_index(),on='work_id')
work_edges_dist_mean_monthly = work_edges_dist_mean.groupby('publication_date_1').dist.mean().to_frame().reset_index()
my_file = "work_edges_dist_mean.csv"    
work_edges_dist_mean.to_csv(os.path.join(path_2authors, my_file),index=False)

In [ ]:
my_file = "work_edges_dist_mean_monthly.csv"     
work_edges_dist_mean_monthly.to_csv(os.path.join(path_2authors, my_file),index=False) 

In [ ]:
df_data1 = df_data1_(work_authors_edges_df_dist)
df_data1['publication_date_1'] = df_data1['publication_date_1'].apply(pd.to_datetime)

### Power-law

In [ ]:
def my_weight(G, u, v, weight="weight"):
    w = 0
    for nbr in set(G[u]) & set(G[v]):
        w += (G[u][nbr].get(weight, 1) + G[v][nbr].get(weight, 1))/2
    return w

def make_institution_graph(works_authors_rows):

    institution_id_set = set(works_authors_rows.institution_id)

    bip_g = nx.from_pandas_edgelist(
        works_authors_rows,
        source='work_id', target='institution_id', edge_attr ='weight'
    )

    inst_graph = nx.bipartite.generic_weighted_projected_graph(bip_g,nodes=institution_id_set,weight_function=my_weight)    
    #inst_graph = nx.bipartite.weighted_projected_graph(bip_g,nodes=institution_id_set) 

    return inst_graph

def fit_data(n,n_authors,my_path_):
    if n==0:
        my_file_add = ""
    else:
        my_file_add = "_tail"+str(n)
        
    my_file = "inst_set.pickle"
    with open(os.path.join(path_2authors, my_file),"rb") as fp:
        inst_set = pickle.load(fp)
    my_file = "dfs_"+str(n_authors)+"authors.pickle"
    with open(os.path.join(my_path_, my_file),"rb") as fp:
        [works_2authors,works,works_authors_aff] = pickle.load(fp) 
    if n>0:
        works = works.loc['2000-01-01':'2023-12-01'] 
        works_authors_aff = works_authors_aff.loc['2000-01-01':'2023-12-01']
        inst_set = set(works_authors_aff.institution_id)
        N_dict = works_authors_aff.groupby('publication_date_1').work_id.nunique().to_frame().to_dict()['work_id']
        months_list = list(N_dict.keys())
        months_list.sort()
        months_list = [i.strftime('%Y-%m-%d') for i in months_list]
        start_index = 0
        end_index = 120 #180
        start_month = months_list[start_index]#.strftime('%Y-%m-%d')
        end_month = months_list[end_index-1]#.strftime('%Y-%m-%d')
        df_TW = works_authors_aff.loc[start_month:end_month] #df_TW = works_authors_aff.loc[months_list[:120]]
        df_ = df_TW.groupby('institution_id').work_id.count().to_frame().sort_values(by='work_id',ascending=False).reset_index().rename(columns={'work_id':'strength'})
        inst_set_s = set((df_[df_.strength>=n]).institution_id)
        #restrict to papers not including authors from institutions outside sample
        inst_set_s_compl = inst_set - inst_set_s
        works_outside = set(works_authors_aff[works_authors_aff.institution_id.isin(inst_set_s_compl)].work_id)
        works = works[~works.work_id.isin(works_outside)]
        works_authors_aff = works_authors_aff[~works_authors_aff.work_id.isin(works_outside)]

    works = works.reset_index()
    works['publication_year'] = works['publication_date_1'].dt.year    
    works_yearly_count = works.groupby('publication_year').work_id.count().to_frame()#.reset_index()
    works = works.set_index('publication_year')
    works_authors_aff = works_authors_aff.drop_duplicates(['work_id','institution_id']).reset_index()
    works_authors_aff['publication_year'] = works_authors_aff['publication_date_1'].dt.year
    works_authors_aff['publication_year'] = works_authors_aff['publication_year'].astype('int64')
    works_authors_aff['institution_id'] = works_authors_aff['institution_id'].astype('int64')
    works_authors_aff = works_authors_aff.sort_values(by='publication_year')
    works_authors_aff['num_affs'] = works_authors_aff.groupby('work_id')['institution_id'].transform('size')
    print(f'{min(works_authors_aff.num_affs)}-{max(works_authors_aff.num_affs)} min-max number (unique) affiliations per work')
    works_authors_aff['weight'] = 2 / ( works_authors_aff['num_affs']*(works_authors_aff['num_affs']-1) ) 
    works_authors_aff.loc[works_authors_aff.num_affs==1,'weight'] = 1 #one affiliation

    works_authors_aff_noloops = works_authors_aff[works_authors_aff.num_affs>1]
    works_authors_aff_loops = works_authors_aff[works_authors_aff.num_affs==1]
    works_authors_aff_loops['institution_id2'] = works_authors_aff_loops['institution_id']
    works_authors_aff_noloops = works_authors_aff_noloops.set_index('publication_year')
    works_authors_aff_loops = works_authors_aff_loops.set_index('publication_year')
    
    I_dist = read_parquet(basepath / 'I_dist_threshold')
    I_dist = I_dist[['source','target','dist']].reset_index(drop=True)

    years_list = list(set(works_authors_aff.publication_year))
    years_list.sort()
    
    works_yearly_count = works_yearly_count.loc[2000:2023]
    print(f"Total number preprints (from {min(works_yearly_count.index)} to {max(works_yearly_count.index)}): {works_yearly_count.work_id.sum()}")
    start_year_0 = 2000
    end_year_0 = 2009
    works_yearly_count_0 = works_yearly_count.loc[start_year_0:end_year_0]
    print(f"Number preprints (from {start_year_0} to {end_year_0}): {works_yearly_count_0.work_id.sum()}")
    start_year_1 = 2010
    end_year_1 = 2014
    works_yearly_count_1 = works_yearly_count.loc[start_year_1:end_year_1]
    print(f"Number preprints (from {start_year_1} to {end_year_1}): {works_yearly_count_1.work_id.sum()}")
    start_year_2 = 2015
    end_year_2 = 2019
    works_yearly_count_2 = works_yearly_count.loc[start_year_2:end_year_2]
    print(f"Number preprints (from {start_year_2} to {end_year_2}): {works_yearly_count_2.work_id.sum()}")
    start_year_3 = 2020
    end_year_3 = 2021
    works_yearly_count_3 = works_yearly_count.loc[start_year_3:end_year_3]
    print(f"Number preprints (from {start_year_3} to {end_year_3}): {works_yearly_count_3.work_id.sum()}")
    start_year_4 = 2022
    end_year_4 = 2023
    works_yearly_count_4 = works_yearly_count.loc[start_year_4:end_year_4]
    print(f"Number preprints (from {start_year_4} to {end_year_4}): {works_yearly_count_4.work_id.sum()}")
    
    works_0 = works.loc[start_year_0:end_year_0]
    works_set_0 = set(works_0.work_id)
    works_1 = works.loc[start_year_1:end_year_1]
    works_set_1 = set(works_1.work_id)
    works_2 = works.loc[start_year_2:end_year_2]
    works_set_2 = set(works_2.work_id)
    works_3 = works.loc[start_year_3:end_year_3]
    works_set_3 = set(works_3.work_id)
    works_4 = works.loc[start_year_4:end_year_4]
    works_set_4 = set(works_4.work_id)

    works_authors_aff_noloops_0 = works_authors_aff_noloops.loc[start_year_0:end_year_0]
    works_authors_aff_loops_0 = works_authors_aff_loops.loc[start_year_0:end_year_0]
    works_authors_aff_noloops_1 = works_authors_aff_noloops.loc[start_year_1:end_year_1]
    works_authors_aff_loops_1 = works_authors_aff_loops.loc[start_year_1:end_year_1]
    works_authors_aff_noloops_2 = works_authors_aff_noloops.loc[start_year_2:end_year_2]
    works_authors_aff_loops_2 = works_authors_aff_loops.loc[start_year_2:end_year_2]
    works_authors_aff_noloops_3 = works_authors_aff_noloops.loc[start_year_3:end_year_3]
    works_authors_aff_loops_3 = works_authors_aff_loops.loc[start_year_3:end_year_3]
    works_authors_aff_noloops_4 = works_authors_aff_noloops.loc[start_year_4:end_year_4]
    works_authors_aff_loops_4 = works_authors_aff_loops.loc[start_year_4:end_year_4]

    works_authors_aff_noloops_0['period'] = 0
    works_authors_aff_noloops_1['period'] = 1
    works_authors_aff_noloops_2['period'] = 2
    works_authors_aff_noloops_3['period'] = 3
    works_authors_aff_noloops_4['period'] = 4
    works_authors_aff_loops_0['period'] = 0
    works_authors_aff_loops_1['period'] = 1
    works_authors_aff_loops_2['period'] = 2
    works_authors_aff_loops_3['period'] = 3
    works_authors_aff_loops_4['period'] = 4
    works_authors_aff_noloops_periods = pd.concat([works_authors_aff_noloops_0,works_authors_aff_noloops_1])
    works_authors_aff_noloops_periods = pd.concat([works_authors_aff_noloops_periods,works_authors_aff_noloops_2])
    works_authors_aff_noloops_periods = pd.concat([works_authors_aff_noloops_periods,works_authors_aff_noloops_3])
    works_authors_aff_noloops_periods = pd.concat([works_authors_aff_noloops_periods,works_authors_aff_noloops_4])
    works_authors_aff_loops_periods = pd.concat([works_authors_aff_loops_0,works_authors_aff_loops_1])
    works_authors_aff_loops_periods = pd.concat([works_authors_aff_loops_periods,works_authors_aff_loops_2])
    works_authors_aff_loops_periods = pd.concat([works_authors_aff_loops_periods,works_authors_aff_loops_3])
    works_authors_aff_loops_periods = pd.concat([works_authors_aff_loops_periods,works_authors_aff_loops_4])

    for y in tqdm(range(5)):
        noloops_df = works_authors_aff_noloops_periods.query("period == @y")
        loops_df = works_authors_aff_loops_periods.query("period == @y")
        loops_df['weight'] = loops_df[['institution_id','institution_id2','weight']].groupby(['institution_id']).weight.transform('sum')
        loops_df = loops_df.drop_duplicates('institution_id')
        I = make_institution_graph(noloops_df)
        I.add_weighted_edges_from([tuple(r) for r in loops_df[['institution_id','institution_id2','weight']].to_numpy()])
        my_file = "I_period"+str(y)+my_file_add+".pickle"
        pickle.dump(I, open(os.path.join(my_path_, my_file), 'wb'))             
        #only loops
        I_loops = nx.Graph()
        I_loops.add_weighted_edges_from([tuple(r) for r in loops_df[['institution_id','institution_id2','weight']].to_numpy()])
        my_file = "Iloops_period"+str(y)+my_file_add+".pickle"
        pickle.dump(I_loops, open(os.path.join(my_path_, my_file), 'wb'))

    info_df = pd.DataFrame()
    degree = {}
    for y in tqdm(range(5)):
        my_file = "I_period"+str(y)+my_file_add+".pickle"
        with open(os.path.join(my_path_, my_file),"rb") as fp:
            I_year = pickle.load(fp)
        my_file = "Iloops_period"+str(y)+my_file_add+".pickle"
        with open(os.path.join(my_path_, my_file),"rb") as fp:
            Iloops_year = pickle.load(fp)
        info_y = pd.DataFrame.from_dict({'period':[y], 'N': [I_year.number_of_nodes()], 'E': [I_year.size()], 'W' : [I_year.size(weight='weight')], 'Nloops': [Iloops_year.number_of_nodes()], 'Eloops': [Iloops_year.size()], 'Wloops' : [Iloops_year.size(weight='weight')]})
        info_df = pd.concat([info_df,info_y])    
        degree[y] = {'k': I_year.degree(), 's' : I_year.degree(weight='weight'),'kloops': Iloops_year.degree(), 'sloops' : Iloops_year.degree(weight='weight')}
    my_file = "info_period_df"+my_file_add+".csv"
    info_df.to_csv(os.path.join(my_path_, my_file),index=False)    
    my_file = "degree_period"+my_file_add+".pickle"
    pickle.dump(degree, open(os.path.join(my_path_, my_file), 'wb'))

    degree_df = pd.DataFrame()
    for y in tqdm(range(5)):
        dict_y = degree[y]
        df_k = pd.DataFrame.from_dict(dict_y['k']).rename(columns={0:'institution_id',1:'k'})
        df_s = pd.DataFrame.from_dict(dict_y['s']).rename(columns={0:'institution_id',1:'s'})
        df_kloops = pd.DataFrame.from_dict(dict_y['kloops']).rename(columns={0:'institution_id',1:'kloops'})
        df_sloops = pd.DataFrame.from_dict(dict_y['sloops']).rename(columns={0:'institution_id',1:'sloops'})
        if len(df_kloops)!=0:
            df_year = (df_k.merge(df_s,on='institution_id')).merge(df_kloops.merge(df_sloops,on='institution_id'),on='institution_id',how='left')
        else:
            df_year = df_k.merge(df_s,on='institution_id')
            df_year["kloops"] = np.nan
            df_year["sloops"] = np.nan
        df_year.insert(0, 'period', y)
        degree_df = pd.concat([degree_df,df_year])
    degree_df = degree_df.fillna(0)
    my_file = "degree_df"+my_file_add+".csv"    
    degree_df.to_csv(os.path.join(my_path_, my_file),index=False) 

    edges_df = pd.DataFrame()
    for y in tqdm(range(5)):
        my_file = "I_period"+str(y)+my_file_add+".pickle"
        with open(os.path.join(my_path_, my_file),"rb") as fp:
            I_year = pickle.load(fp)
        I_year_df = nx.to_pandas_edgelist(I_year)
        I_year_df = I_year_df.merge(I_dist,on=['source','target'],how='left')
        I_year_df.insert(0, 'period', y)
        edges_df = pd.concat([edges_df,I_year_df])
    my_file = "edges_df"+my_file_add+".csv"    
    edges_df.to_csv(os.path.join(my_path_, my_file),index=False) 

    edges_comp_df = pd.DataFrame()
    for y in tqdm(range(5)):
        my_file = "I_period"+str(y)+my_file_add+".pickle" 
        with open(os.path.join(my_path_, my_file),"rb") as fp:
            I = pickle.load(fp)
        I_comp = nx.complement(I) #20 mins
        # my_file = "I_period"+str(y)+"_comp.pickle"
        # pickle.dump(I_comp, open(os.path.join(my_path_, my_file), 'wb')) 
        I_comp_df = nx.to_pandas_edgelist(I_comp)
        I_comp_df = I_comp_df.merge(I_dist,on=['source','target'])
        I_comp_df.insert(0, 'period', y)
        edges_comp_df = pd.concat([edges_comp_df,I_comp_df])    
    my_file = "edges_comp_df"+my_file_add+".csv"    
    edges_comp_df.to_csv(os.path.join(my_path_, my_file),index=False)  

In [ ]:
fit_data(0,2,path_2authors)  

In [ ]:
fit_data(20,2,path_2authors)  

In [ ]:
fit_data(50,2,path_2authors)  

In [ ]:
def plots(n,my_path_):
    if n==0:
        my_file_add = ""
    else:
        my_file_add = "_tail"+str(n)    
    
    periods_list = [0,1,2,3,4]
    periods_labels = ['[2000,2009]','[2010,2014]','[2015,2019]','[2020,2021]','[2022,2023]']
    color_dict = {0: 'yellow', 1: 'magenta', 2: 'mediumseagreen', 3: 'cyan', 4: 'orange'}
    
    my_file = "edges_df"+my_file_add+".csv"
    edges_df = pd.read_csv(my_path_ / my_file) 
    my_file = "degree_df"+my_file_add+".csv"
    degree_df = pd.read_csv(my_path_ / my_file)  
    my_file = "edges_comp_df"+my_file_add+".csv"
    edges_comp_df = pd.read_csv(my_path_ / my_file) 
    edges_comp_df['weight'] = 0
    edges_df_tot = pd.concat([edges_df,edges_comp_df])
    
    def mean_scatter_plot_multi(periods_list,periods_labels,color_dict,func,edges_df,degree_df,col,columnx,columny,labelx,labely,title,logx=False,logy=False,limity=False,geomspace=False):
        fig, ax = plt.subplots(figsize=(8,5))
        for y in periods_list[0:]:
            df1 = func(edges_df,degree_df,col,y)
            x1 = np.array(df1[columnx]) 
            y1 = np.array(df1[columny])  
            # Define the grid
            gridsize = 10**2
            if geomspace:
                xbins1 = np.geomspace(x1.min(), x1.max(), gridsize)
                #xbins2 = np.geomspace(x2.min(), x2.max(), gridsize)
            else:
                xbins1 = np.linspace(x1.min(), x1.max(), gridsize)
                #xbins2 = np.linspace(x2.min(), x2.max(), gridsize)
            # Calculate the mean values within each column
            mean_values1 = []
            for i in range(len(xbins1) - 1):
                mask = (x1 >= xbins1[i]) & (x1 < xbins1[i + 1])
                mean_y1 = np.mean(y1[mask])
                mean_values1.append(mean_y1) 
            ax.plot((xbins1[:-1] + xbins1[1:]) / 2, mean_values1, '.',color=color_dict[y],markersize=5, label=periods_labels[y])          
        ax.set_xlabel(labelx,size=20)
        ax.set_ylabel(labely,size=20)
        ax.set_title(title,size=30)
        if logx==True:
            ax.set_xscale('log')  
        if logy==True:
            ax.set_yscale('log')
        if limity==True:
            ax.set_ylim([0, 1])  
        ax.legend(bbox_to_anchor=(1.0, 0.9), prop={'size': 15}, markerscale=3)
        plt.show() 
    def scipy_fun_multi(periods_list,periods_labels,color_dict,func,edges_df,degree_df,col,x_max,labelx,labely,title):
        fig, ax = plt.subplots(figsize=(8,5))  
        for y in periods_list[0:]:
            x1,y1 = func(edges_df,degree_df,col,y)
            y1 = np.array(y1)
            popt1, pcov1 = scipy.optimize.curve_fit(linear, x1[x1<=x_max], y1[x1<=x_max])
            perr1 = np.sqrt(np.diag(pcov1))
            print(f'{periods_labels[y]}: gamma {popt1[0]} (perr {perr1[0]:.4f})') 
            ax.scatter(x1[x1<=x_max], y1[x1<=x_max],marker='.',color=color_dict[y],linewidths=0.01, label=periods_labels[y]) 
            ax.plot(x1[x1<=x_max], linear(x1[x1<=x_max], *popt1),color=color_dict[y],linewidth=2.0)   
        ax.set_xlabel(labelx,size=20)
        ax.set_ylabel(labely,size=20)
        ax.legend(bbox_to_anchor=(1.0, 0.9), prop={'size': 15}, markerscale=3)
        ax.set_title(title+" - linear",size=30)
        plt.show()
    def linear(x, b):
        return  b * x 
    def linear_fit(edges_df,degree_df,col,yy):
        df1 = fig_2B(edges_df,degree_df,col,yy)
        x1 = np.array(df1["s"]) 
        y1 = np.array(df1[col])  
        gridsize = 10**2
        xbins1 = np.linspace(x1.min(), x1.max(), gridsize)    
        mean_values1 = []
        for i in range(len(xbins1) - 1):
            mask = (x1 >= xbins1[i]) & (x1 < xbins1[i + 1])
            mean_y1 = np.mean(y1[mask])
            mean_values1.append(mean_y1)     
        x = (xbins1[:-1] + xbins1[1:]) / 2
        y = np.array(mean_values1)
        x = x[~np.isnan(y)]
        y = y[~np.isnan(y)]
        return x,y
    
    def fig_2B(edges_df,degree_df,col,y):
        edges_year_df = edges_df.query('period == @y')
        loops_w_count = edges_year_df[edges_year_df.source == edges_year_df.target][['source',col]].rename(columns={'source':'institution_id'})
        #degree_year_df = (degree_df[degree_df.period<=y][['institution_id','s']]).groupby(['institution_id']).s.sum().to_frame().reset_index()
        degree_year_df = degree_df.query('period == @y')
        loops_w_count = loops_w_count.merge(degree_year_df,on='institution_id')
        return loops_w_count
    
    mean_scatter_plot_multi(periods_list,periods_labels,color_dict,fig_2B,edges_df_tot,degree_df,"weight","s","weight","s_i","w_ii",'Fig. 2B',False,False,False,False)
    scipy_fun_multi(periods_list,periods_labels,color_dict,linear_fit,edges_df_tot,degree_df,"weight",2*10**4,"s_i","w_ii","Fig. 2B")

    def fig_4B(df1,df2,col,y):
        df1 = df1.query('period == @y')[['source','target',col,'dist']]
        df2 = df2.query('period == @y')[['institution_id','s']]
        df2_dict = df2.set_index('institution_id').to_dict()['s']
        df1['source_s'] = df1['source'].map(df2_dict)
        df1['target_s'] = df1['target'].map(df2_dict)
        df1['ratio'] = df1[col] / (df1['source_s']*df1['target_s'])**(1/2)
        df1 = df1[["dist", "ratio"]]
        #delate zero
        df1 = df1[df1.dist>0]
        return df1
    def scipy_fun_multi(periods_list,periods_labels,color_dict,func,edges_df,degree_df,col,x_min,labelx,labely,title):
        fig, ax = plt.subplots(figsize=(8,5))  
        for y in periods_list[0:]:
            x1,y1 = func(edges_df,degree_df,col,y)
            y1 = np.array(y1)
            popt1, pcov1 = scipy.optimize.curve_fit(power2, x1[x1>=x_min], y1[x1>=x_min])
            perr1 = np.sqrt(np.diag(pcov1))
            print(f'{periods_labels[y]}: alpha {(-1)*popt1[1]:.3f} (perr {perr1[1]:.4f}), beta {popt1[0]:.6f} (perr {perr1[0]:.4f})') 
            ax.scatter(x1[x1>=x_min], y1[x1>=x_min],marker='.',color=color_dict[y],linewidths=0.01, label=periods_labels[y]) 
            ax.plot(x1[x1>=x_min], power2(x1[x1>=x_min], *popt1),color=color_dict[y],linewidth=2.0)   
        ax.set_xscale('log')  
        ax.set_yscale('log')
        ax.set_xlabel(labelx,size=20)
        ax.set_ylabel(labely,size=20)
        ax.legend(bbox_to_anchor=(1.0, 0.9), prop={'size': 15}, markerscale=3)
        ax.set_title(title+" - power law",size=30)
        plt.show()
    def power2(x, b, c):
        return  b * x ** c 
    def powerlaw_fit(edges_df,degree_df,col,yy):
        df = fig_4B(edges_df,degree_df,col,yy)
        x = np.array(df["dist"]) 
        y = np.array(df["ratio"])
        gridsize = 10**2
        xbins = np.geomspace(x.min(), x.max(), gridsize)
        # Calculate the mean values within each column
        mean_values = []
        for i in range(len(xbins) - 1):
            mask = (x >= xbins[i]) & (x < xbins[i + 1])
            mean_y = np.mean(y[mask])
            mean_values.append(mean_y) 
        x = (xbins[:-1] + xbins[1:]) / 2
        y = np.array(mean_values)
        return x,y
    
    mean_scatter_plot_multi(periods_list,periods_labels,color_dict,fig_4B,edges_df_tot,degree_df,"weight","dist", "ratio","d_ij","w_ij / (s_i*s_j)^(1/2) ",'Fig. 4B',True,True,False,True)
    scipy_fun_multi(periods_list,periods_labels,color_dict,powerlaw_fit,edges_df_tot,degree_df,"weight",10**1,"d_ij","w_ij / (s_i*s_j)^(1/2) ","Fig. 4B - power law")
    scipy_fun_multi(periods_list,periods_labels,color_dict,powerlaw_fit,edges_df_tot,degree_df,"weight",10**2,"d_ij","w_ij / (s_i*s_j)^(1/2) ","Fig. 4B - power law")

In [ ]:
plots(0,path_2authors)

In [ ]:
plots(20,path_2authors)

In [ ]:
plots(50,path_2authors)

## Edges - 3 authors

In [ ]:
path_3authors = Path('./Model_3authors') 
if not os.path.exists(path_3authors):
    os.makedirs(path_3authors)

### Tables

In [ ]:
works = read_parquet(basepath / 'works')
works_authors_aff = read_parquet(basepath / 'works_authors_aff')

works = works.loc['2000-01-01':'2023-12-01'] 
works = works[works.num_authors>1]

works_3authors = set(works[works.num_authors ==3].work_id)
print(f'{len(works_3authors)} ({(len(works_3authors)/len(works))*100:.2f}%) works 3 authors')

works = works[works.work_id.isin(works_3authors)]
works_authors_aff = works_authors_aff[works_authors_aff.work_id.isin(works_3authors)]

my_file = "dfs_3authors.pickle"
pickle.dump([works_3authors,works,works_authors_aff], open(os.path.join(path_3authors, my_file), 'wb')) 

In [ ]:
#INITIALIZATION #preferential attachment  #TW: 2000-2009
my_file = "dfs_3authors.pickle"
with open(os.path.join(path_3authors, my_file),"rb") as fp:
    [works_3authors,works,works_authors_aff] = pickle.load(fp)  
works = works.loc['2000-01-01':'2023-12-01'] 
works_authors_aff = works_authors_aff.loc['2000-01-01':'2023-12-01']

N_dict = works_authors_aff.groupby('publication_date_1').work_id.nunique().to_frame().to_dict()['work_id']
months_list = list(N_dict.keys())
months_list.sort()
months_list = [i.strftime('%Y-%m-%d') for i in months_list]

start_index = 0
end_index = 120 #180
start_month = months_list[start_index]#.strftime('%Y-%m-%d')
end_month = months_list[end_index-1]#.strftime('%Y-%m-%d')

df_TW = works_authors_aff.loc[start_month:end_month] #df_TW = works_authors_aff.loc[months_list[:120]]
inst_set = set(df_TW.institution_id)
I = len(inst_set)
print(f'TW from {start_month} to {end_month} : {I} institutions')
# my_file = "inst_set.pickle"
# pickle.dump(inst_set, open(os.path.join(path_3authors, my_file), 'wb'))

In [ ]:
#initial strenghts    #consider only institutions in sample in TW
#count number (unique) institutions per paper
df_TW = df_TW.drop_duplicates(['work_id','institution_id'])
df_TW['num_affs'] = df_TW.groupby('work_id')['institution_id'].transform('size')
print(f'{min(df_TW.num_affs)}-{max(df_TW.num_affs)} min-max number (unique) affiliations per work')
df_TW['weight'] = 2 / ( df_TW['num_affs']*(df_TW['num_affs']-1) ) 
df_TW.loc[df_TW.num_affs==1,'weight'] = 1 #one affiliation
df_TW_noloops = df_TW[df_TW.num_affs>1]
df_TW_loops = df_TW[df_TW.num_affs==1]
df_TW_loops['institution_id2'] = df_TW_loops['institution_id']
df_TW_loops['weight'] = df_TW_loops[['institution_id','institution_id2','weight']].groupby(['institution_id']).weight.transform('sum')
df_TW_loops = df_TW_loops.drop_duplicates('institution_id')
I_graph = make_institution_graph(df_TW_noloops)
I_graph.add_weighted_edges_from([tuple(r) for r in df_TW_loops[['institution_id','institution_id2','weight']].to_numpy()])
df_ = pd.DataFrame.from_dict(dict(I_graph.degree(weight='weight')),orient='index').reset_index().rename(columns={'index':'institution_id',0:'strength'})
df_['institution_id'] = df_['institution_id'].astype(int)
df = df_.sort_values(by='strength',ascending=False)
inst_set = set(df.institution_id)
I = len(inst_set)
print(f'TW from {start_month} to {end_month} : {I} institutions')

my_file = "df_strengths0.csv"     
df.to_csv(os.path.join(path_3authors, my_file),index=False)

my_file = "inst_set.pickle"
pickle.dump(inst_set, open(os.path.join(path_3authors, my_file), 'wb'))

In [ ]:
I_dist = read_parquet(basepath / 'I_dist_threshold')
I_dist = I_dist[['source','target','dist']].reset_index(drop=True)
I_dist_sort = [tuple(row) for row in I_dist.itertuples(index=False)]
I_dist_sort = [ sorted(list(x)[:2])+[list(x)[2]] for x in I_dist_sort]
I_dist_sort = pd.DataFrame(I_dist_sort, columns = ['source', 'target', 'dist'])
my_file = "I_dist_model.csv" 
I_dist_sort.to_csv(os.path.join(path_3authors, my_file),index=False)

### Data plots

In [ ]:
my_file = "dfs_3authors.pickle"
with open(os.path.join(path_3authors, my_file),"rb") as fp:
    [works_3authors,works,works_authors_aff] = pickle.load(fp)  

N_dict = works_authors_aff.groupby('publication_date_1').work_id.nunique().to_frame().to_dict()['work_id']

months_list = list(N_dict.keys())
months_list.sort()
months_list = [i.strftime('%Y-%m-%d') for i in months_list]

In [ ]:
def plot_fit_rolling_breakpoints_2(df,x_column,x_column2,x_label,title,window_size,num_breakpoints):

    fig, ax = plt.subplots(figsize=(15, 5))
    x_dates = list(df['publication_date_1'])
    y_data = df[x_column].rolling(window=window_size).mean()[window_size-1:]
    y_data2 = df[x_column2].rolling(window=window_size).mean()[window_size-1:]
    x_data = x_dates[window_size-1:]
    ax.set_ylim([0, 1])
    
    ax.plot(x_data, y_data, "o-", color='orange', markersize=3,label='intra-institution')
    ax.plot(x_data, y_data2, "o-", color='green', markersize=3,label='inter-institution')

    plt.grid(True, linewidth=0.5)
    ax.yaxis.set_major_formatter(formatter)

    ax.set_xlabel(x_label,size=20)
    #ax.set_title(title,size=30)

    #ax.xaxis.set_major_locator(mdates.MonthLocator()) # Make ticks on occurrences of each month
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %y')) # Get only the month to show in the x-axis

    ax.axvline(pd.Timestamp(2020, 3, 1),color='r') #ax.axvline(x_dates[84],color='r')

    #save for all possible combination of breakpoints the correspondent error 
    def f(breakpoints, x, y, fcache): 
        breakpoints = tuple(map(int, sorted(breakpoints)))
        if breakpoints not in fcache:
            total_error = 0
            for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x, y):
                total_error += ((f(xi) - yi)**2).sum()
            fcache[breakpoints] = total_error 
        # print('{} --> {}'.format(breakpoints, fcache[breakpoints]))
        return fcache[breakpoints]

    def find_best_piecewise_polynomial(breakpoints, x, y):
        breakpoints = tuple(map(int, sorted(breakpoints)))
        xs = np.split(x, breakpoints)
        ys = np.split(y, breakpoints)
        result = []
        for xi, yi in zip(xs, ys):
            if len(xi) < 2: continue
            coefs = npoly.polyfit(xi, yi, 1)
            f = npoly.Polynomial(coefs)
            result.append([f, xi, yi])
        return result
    
    x_num = dates.date2num(x_data)
    breakpoints = optimize.brute(f, [slice(1, len(x_data), 1)]*num_breakpoints, args=(x_num, y_data, {}), finish=None)
    if num_breakpoints==1:
        breakpoints = [breakpoints] 
    for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x_num, y_data):
        xi_min = x_data[np.where(x_num == xi.min())[0][0]]
        xi_max = x_data[np.where(x_num == xi.max())[0][0]]
        x_interval = np.array([xi_min, xi_max]) 
        #print('y = {:35s}, if x in [{}, {}]'.format(str(f), *x_interval))
        x_interval = np.array([xi.min(), xi.max()])
        #ax.plot(x_interval, f(x_interval), 'yo-')
        coef = f.convert().coef
        b = coef[0]
        a = coef[1]
        if b>0:
            ll = 'y = {:.2f} x + {:.0f}'.format(a,b)
        else: #b<0
            ll = 'y = {:.2f} x - {:.0f}'.format(a,abs(b))    
        ax.plot(x_interval, f(x_interval), 'yo-',linewidth=2, markersize=7)       
    #save for all possible combination of breakpoints the correspondent error
    
    def f(breakpoints, x, y, fcache): 
        breakpoints = tuple(map(int, sorted(breakpoints)))
        if breakpoints not in fcache:
            total_error = 0
            for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x, y):
                total_error += ((f(xi) - yi)**2).sum()
            fcache[breakpoints] = total_error 
        # print('{} --> {}'.format(breakpoints, fcache[breakpoints]))
        return fcache[breakpoints]

    def find_best_piecewise_polynomial(breakpoints, x, y):
        breakpoints = tuple(map(int, sorted(breakpoints)))
        xs = np.split(x, breakpoints)
        ys = np.split(y, breakpoints)
        result = []
        for xi, yi in zip(xs, ys):
            if len(xi) < 2: continue
            coefs = npoly.polyfit(xi, yi, 1)
            f = npoly.Polynomial(coefs)
            result.append([f, xi, yi])
        return result
    
    x_num = dates.date2num(x_data)
    breakpoints = optimize.brute(f, [slice(1, len(x_data), 1)]*num_breakpoints, args=(x_num, y_data2, {}), finish=None)
    if num_breakpoints==1:
        breakpoints = [breakpoints] 
    for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x_num, y_data2):
        xi_min = x_data[np.where(x_num == xi.min())[0][0]]
        xi_max = x_data[np.where(x_num == xi.max())[0][0]]
        x_interval = np.array([xi_min, xi_max]) 
        #print('y = {:35s}, if x in [{}, {}]'.format(str(f), *x_interval))
        x_interval = np.array([xi.min(), xi.max()])
        #ax.plot(x_interval, f(x_interval), 'yo-')
        coef = f.convert().coef
        b = coef[0]
        a = coef[1]
        if b>0:
            ll = 'y = {:.2f} x + {:.0f}'.format(a,b)
        else: #b<0
            ll = 'y = {:.2f} x - {:.0f}'.format(a,abs(b))    
        ax.plot(x_interval, f(x_interval), 'yo-',linewidth=2, markersize=7) 
       
    ax.legend()        
    ax.set_title(title,size=30)

In [ ]:
import matplotlib.dates as dates
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
from scipy import optimize
import numpy.polynomial.polynomial as npoly

def form(x,pos):
    if x<1e3:
        return '%1.3f' % (x)
    elif x<1e6:
        return '%1.1fK' % (x * 1e-3)
    else:
        return '%1.1fM' % (x * 1e-6)
formatter = FuncFormatter(form)

def plot_fit_rolling_breakpoints(df,x_column,x_label,title,window_size,num_breakpoints,ff):
    
    #save for all possible combination of breakpoints the correspondent error 
    def f(breakpoints, x, y, fcache): 
        breakpoints = tuple(map(int, sorted(breakpoints)))
        if breakpoints not in fcache:
            total_error = 0
            for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x, y):
                total_error += ((f(xi) - yi)**2).sum()
            fcache[breakpoints] = total_error 
        # print('{} --> {}'.format(breakpoints, fcache[breakpoints]))
        return fcache[breakpoints]

    def find_best_piecewise_polynomial(breakpoints, x, y):
        breakpoints = tuple(map(int, sorted(breakpoints)))
        xs = np.split(x, breakpoints)
        ys = np.split(y, breakpoints)
        result = []
        for xi, yi in zip(xs, ys):
            if len(xi) < 2: continue
            coefs = npoly.polyfit(xi, yi, 1)
            f = npoly.Polynomial(coefs)
            result.append([f, xi, yi])
        return result

    fig, ax = plt.subplots(figsize=(15, 5))
    x_dates = list(df['publication_date_1'])
    y_data = df[x_column].rolling(window=window_size).mean()[window_size-1:]
    x_data = x_dates[window_size-1:]
    
    ax.plot(x_data, y_data, "o-", color='orange', markersize=3)

    plt.grid(True, linewidth=0.5)
    ax.yaxis.set_major_formatter(formatter)

    ax.set_xlabel(x_label,size=20)
    ax.set_title(title,size=30)

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %y')) # Get only the month to show in the x-axis

    ax.axvline(pd.Timestamp(2020, 3, 1),color='r') #ax.axvline(x_dates[84],color='r')

    x_num = dates.date2num(x_data)
    breakpoints = optimize.brute(f, [slice(1, len(x_data), 1)]*num_breakpoints, args=(x_num, y_data, {}), finish=None)
    if num_breakpoints==1:
        breakpoints = [breakpoints] 

    for f, xi, yi in find_best_piecewise_polynomial(breakpoints, x_num, y_data):
        xi_min = x_data[np.where(x_num == xi.min())[0][0]]
        xi_max = x_data[np.where(x_num == xi.max())[0][0]] 
        x_interval = np.array([xi.min(), xi.max()])
        coef = f.convert().coef
        b = coef[0]
        a = coef[1]
        if ff==1:
            if b>0:
                ll = 'y = {:.2f} x + {:.2f}'.format(a,b)
            else: #b<0
                ll = 'y = {:.2f} x - {:.2f}'.format(a,abs(b))    
        else:
            if b>0:
                ll = 'y = {:.5f} x + {:.5f}'.format(a,b)
            else: #b<0
                ll = 'y = {:.5f} x - {:.5f}'.format(a,abs(b))  
        #x_1 = x_dates[window_size-1:][np.where(x_num==x_interval[0])[0][0]]
        #x_3 = x_dates[window_size-1:][np.where(x_num==x_interval[1])[0][0]]
        ax.plot(x_interval, f(x_interval), 'o-',color='yellow',label=ll, markersize=6)
        
    ax.legend()
    ax.set_title(title,size=30)
    #return x_num 
    
x_label='Month'
window_size = 6
num_breakpoints = 1

In [ ]:
def work_edges_dist_mean_monthly_(works_outside,works_set,path):
    my_file = "work_edges_dist_mean.csv"    
    work_edges_dist_mean = pd.read_csv(os.path.join(path, my_file))
    work_edges_dist_mean = work_edges_dist_mean[work_edges_dist_mean.work_id.isin(works_set)]
    work_edges_dist_mean = work_edges_dist_mean[~work_edges_dist_mean.work_id.isin(works_outside)]
    work_edges_dist_mean['publication_date_1'] = pd.to_datetime(work_edges_dist_mean['publication_date_1'])
    work_edges_dist_mean_monthly = work_edges_dist_mean.groupby('publication_date_1').dist.mean().to_frame().reset_index()
    return work_edges_dist_mean_monthly

def work_edges_dist_max_monthly_(works_outside,works_set,path):
    my_file = "work_edges_dist_max.csv"    
    work_edges_dist_mean = pd.read_csv(os.path.join(path, my_file))
    work_edges_dist_mean = work_edges_dist_mean[work_edges_dist_mean.work_id.isin(works_set)]
    work_edges_dist_mean = work_edges_dist_mean[~work_edges_dist_mean.work_id.isin(works_outside)]
    work_edges_dist_mean['publication_date_1'] = pd.to_datetime(work_edges_dist_mean['publication_date_1'])
    work_edges_dist_mean_monthly = work_edges_dist_mean.groupby('publication_date_1').dist.mean().to_frame().reset_index()
    return work_edges_dist_mean_monthly

In [ ]:
work_authors_edges_df_dist = read_parquet(Path('./TeamDistance') / 'work_authors_edges_df_dist')
work_authors_edges_df_dist = works[['work_id']].reset_index().merge(work_authors_edges_df_dist,on='work_id')
work_authors_edges_df_dist = work_authors_edges_df_dist[work_authors_edges_df_dist.work_id.isin(works_3authors)]
work_edges_dist_mean = work_authors_edges_df_dist.groupby('work_id').dist.mean().to_frame().reset_index()
work_edges_dist_mean = work_edges_dist_mean.merge(works[['work_id']].reset_index(),on='work_id')
work_edges_dist_mean_monthly = work_edges_dist_mean.groupby('publication_date_1').dist.mean().to_frame().reset_index()
my_file = "work_edges_dist_mean.csv"    
work_edges_dist_mean.to_csv(os.path.join(path_3authors, my_file),index=False)

In [ ]:
my_file = "work_edges_dist_mean_monthly.csv"     
work_edges_dist_mean_monthly.to_csv(os.path.join(path_3authors, my_file),index=False) 

In [ ]:
my_file = "dfs_3authors.pickle"
with open(os.path.join(path_3authors, my_file),"rb") as fp:
    [works_3authors,works_3authors,works_authors_aff_3authors] = pickle.load(fp)  
inst_set_whole_3authors = set((works_authors_aff_3authors.drop_duplicates('institution_id')).institution_id)    

In [ ]:
N_dict = works_authors_aff_3authors.groupby('publication_date_1').work_id.nunique().to_frame().to_dict()['work_id']
months_list = list(N_dict.keys())
months_list.sort()
months_list = [i.strftime('%Y-%m-%d') for i in months_list]
start_index = 0
end_index = 120 
start_month = months_list[start_index]
end_month = months_list[end_index-1]
works_authors_aff_3authors_TW = works_authors_aff_3authors.loc[start_month:end_month]

inst_set_TW_3authors = set((works_authors_aff_3authors_TW .drop_duplicates('institution_id')).institution_id)

In [ ]:
#restrict to papers not including authors from institutions not in TW
inst_set_TW_3authors_compl = inst_set_whole_3authors - inst_set_TW_3authors
works_outside = set(works_authors_aff_3authors[works_authors_aff_3authors.institution_id.isin(inst_set_TW_3authors_compl)].work_id)
works_3authors = works_3authors[~works_3authors.work_id.isin(works_outside)]
works_authors_aff_3authors_new = works_authors_aff_3authors[~works_authors_aff_3authors.work_id.isin(works_outside)]

In [ ]:
print(f'3 authors. [2000,2023] {len(inst_set_whole_3authors)} institutions, [2000,2009] {len(inst_set_TW_3authors)} institutions ({(len(inst_set_TW_3authors)/len(inst_set_whole_3authors))*100:.2f}%)')

In [ ]:
print(f'3 authors. All institutions: {len(set(works_authors_aff_3authors.work_id))} works, restrict to institutions in TW {len(set(works_authors_aff_3authors_new.work_id))} works ({(len(set(works_authors_aff_3authors_new.work_id))/len(set(works_authors_aff_3authors.work_id)))*100:.2f}%)')

In [ ]:
#delate tail
df_strengths_3authors_TW = works_authors_aff_3authors_TW.groupby('institution_id').work_id.count().to_frame().sort_values(by='work_id',ascending=False).reset_index().rename(columns={'work_id':'strength'})
df_strengths_3authors_TW['institution_id'] = df_strengths_3authors_TW['institution_id'].astype(int)

In [ ]:
for n in [20,50,70,100]:
    my_file = "dfs_3authors.pickle"
    with open(os.path.join(path_3authors, my_file),"rb") as fp:
        [works_3authors,works_3authors,works_authors_aff_3authors] = pickle.load(fp) 
    I = set((df_strengths_3authors_TW[df_strengths_3authors_TW.strength>=n]).institution_id)
    I_compl = inst_set_whole_3authors - I
    works_outside = set(works_authors_aff_3authors[works_authors_aff_3authors.institution_id.isin(I_compl)].work_id)
    works_3authors = works_3authors[~works_3authors.work_id.isin(works_outside)]
    works_authors_aff_3authors_new = works_authors_aff_3authors[~works_authors_aff_3authors.work_id.isin(works_outside)]
    print(f'3 authors - tail {n}. TW {len(I)} institutions ({(len(I)/len(inst_set_TW_3authors))*100:.2f}%), {len(set(works_authors_aff_3authors_new.work_id))} works ({(len(set(works_authors_aff_3authors_new.work_id))/len(set(works_authors_aff_3authors.work_id)))*100:.2f}%)')

### Power-law

In [ ]:
fit_data(0,3,path_3authors) 

In [ ]:
fit_data(20,3,path_3authors) 

In [ ]:
fit_data(50,3,path_3authors) 

In [ ]:
plots(0,path_3authors)

In [ ]:
plots(20,path_3authors)

In [ ]:
plots(50,path_3authors)

In [ ]:
## triangles ## different distances

In [ ]:
n = 50 
my_file = "insts_tail"+str(n)+"_TW.pickle"
with open(os.path.join(path_3authors, my_file),"rb") as fp:
    inst_set_s = pickle.load(fp)
len(inst_set_s)

In [ ]:
lll = range(800)
all_triangles = set(itertools.combinations(list(lll), 3))#all possible triangles i<j<k
all_triangles = pd.DataFrame(all_triangles, columns =['i', 'j', 'k'])

In [ ]:
all_triangles.head(5)

In [ ]:
#800
#1000 #2mins

In [ ]:
#TW
my_file = "dfs_3authors.pickle"
with open(os.path.join(path_3authors, my_file),"rb") as fp:
    [works_3authors,works,works_authors_aff] = pickle.load(fp)  
works = works.loc['2000-01-01':'2023-12-01'] 
works_authors_aff = works_authors_aff.loc['2000-01-01':'2023-12-01']

In [ ]:
len(set(works_authors_aff.institution_id))

In [ ]:
N_dict = works_authors_aff.groupby('publication_date_1').work_id.nunique().to_frame().to_dict()['work_id']
months_list = list(N_dict.keys())
months_list.sort()
months_list = [i.strftime('%Y-%m-%d') for i in months_list]

#INITIALIZATION #preferential attachment  #TW: 2000-2009
start_index = 0
end_index = 120 #180
start_month = months_list[start_index]#.strftime('%Y-%m-%d')
end_month = months_list[end_index-1]#.strftime('%Y-%m-%d')
df_TW = works_authors_aff.loc[start_month:end_month] 

In [ ]:
len(set(df_TW.institution_id))

In [ ]:
my_file = "I_dist_model.csv"  
I_dist = pd.read_csv(os.path.join(path_3authors, my_file))
I_dist = I_dist.query('source.isin(@inst_set_s) & target.isin(@inst_set_s)')

all_triangles = set(itertools.combinations(list(inst_set_s), 3))#all possible triangles i<j<k
all_triangles = [sorted(list(x)) for x in all_triangles]
df_inter3 = pd.DataFrame(all_triangles, columns =['i', 'j', 'k'])

In [ ]:
def power_law_threshold_triangles(n):
    my_file = "insts_tail"+str(n)+"_TW.pickle"
    with open(os.path.join(path_3authors, my_file),"rb") as fp:
        inst_set_s = pickle.load(fp)

    my_file = "I_dist_model.csv"  
    I_dist = pd.read_csv(os.path.join(path_3authors, my_file))
    I_dist = I_dist[['source','target','dist']]
    I_dist = I_dist.query('source.isin(@inst_set_s) & target.isin(@inst_set_s)')
    I_dist['source'] = I_dist['source'].astype(int)
    I_dist['target'] = I_dist['target'].astype(int)
    df_inter2 = I_dist[I_dist.source!=I_dist.target]
    df_inter2['dist_mean'] = df_inter2['dist']*(2/3)
    df_inter2['dist_max'] = df_inter2['dist']
    df_inter2 = df_inter2.drop(columns='dist').reset_index(drop=True)
    df_inter2 = df_inter2.rename(columns={'source':'i','target':'j'})
    df_inter2_copy = df_inter2.copy()
    df_inter2['k'] = df_inter2['i']
    df_inter2_copy['k'] = df_inter2_copy['j']
    df_inter2 = pd.concat([df_inter2,df_inter2_copy])
    df_intra = I_dist[I_dist.source==I_dist.target]
    df_intra['dist_mean'] = 0
    df_intra['dist_max'] = 0
    df_intra['dist_RMS'] = 0
    df_intra = df_intra.drop(columns='dist').reset_index(drop=True)
    df_intra = df_intra.rename(columns={'source':'i','target':'j'})
    df_intra['k']=df_intra['i']
    df_intra = df_intra[['i','j','k','dist_mean','dist_max','dist_RMS']]

    my_file = "df_inter3_tail"+str(n)+".csv" 
    df_inter3 = pd.read_csv(os.path.join(path_3authors, my_file))
    df_inter3['i'] = df_inter3['i'].astype(int)
    df_inter3['j'] = df_inter3['j'].astype(int)
    df_inter3['k'] = df_inter3['k'].astype(int)
    df_inter3 = df_inter3.query('i.isin(@inst_set_s) and j.isin(@inst_set_s) and k.isin(@inst_set_s)')
    df_inter3 = df_inter3.drop(columns='dist')
    df_inter3['dist_mean'] = (df_inter3['d_ij']+df_inter3['d_jk']+df_inter3['d_ik'])/3
    df_inter3['dist_max'] = df_inter3[['d_ij','d_jk','d_ik']].max(axis=1)

    ## Center of mass
    basepath4 = Path('/N/project/openalex/ssikdar/processed-snapshots/csv-files/aug-2023')
    institutions_geo_df = pd.read_csv(basepath4 / 'institutions_geo.csv.gz')
    institutions_geo_df = institutions_geo_df[['institution_id','latitude','longitude']]
    institutions_geo_df = pd.concat([institutions_geo_df,pd.DataFrame.from_dict({'institution_id':4210144721,'latitude':42.48948,'longitude':-83.14465},orient='index').T])
    institutions_geo_df = institutions_geo_df.query('institution_id.isin(@inst_set_s)')
    institutions_geo_df['latitude_rad'] = np.radians(institutions_geo_df['latitude']) # Convert lat/lng from degrees to radians
    institutions_geo_df['longitude_rad'] = np.radians(institutions_geo_df['longitude'])
    R = 6371 # Radius of Earth in kilometers
    # Calculate Cartesian coordinates
    institutions_geo_df['x'] = R * np.cos(institutions_geo_df['latitude_rad']) * np.cos(institutions_geo_df['longitude_rad'])
    institutions_geo_df['y'] = R * np.cos(institutions_geo_df['latitude_rad']) * np.sin(institutions_geo_df['longitude_rad'])
    institutions_geo_df['z'] = R * np.sin(institutions_geo_df['latitude_rad'])
    institutions_geo_mapx = dict(zip(institutions_geo_df['institution_id'],institutions_geo_df['x']))
    institutions_geo_mapy = dict(zip(institutions_geo_df['institution_id'],institutions_geo_df['y']))
    institutions_geo_mapz = dict(zip(institutions_geo_df['institution_id'],institutions_geo_df['z']))

    df_inter2['i_x'] = df_inter2['i'].map(institutions_geo_mapx)
    df_inter2['i_y'] = df_inter2['i'].map(institutions_geo_mapy)
    df_inter2['i_z'] = df_inter2['i'].map(institutions_geo_mapz)
    df_inter2['j_x'] = df_inter2['j'].map(institutions_geo_mapx)
    df_inter2['j_y'] = df_inter2['j'].map(institutions_geo_mapy)
    df_inter2['j_z'] = df_inter2['j'].map(institutions_geo_mapz)
    df_inter2['k_x'] = df_inter2['k'].map(institutions_geo_mapx)
    df_inter2['k_y'] = df_inter2['k'].map(institutions_geo_mapy)
    df_inter2['k_z'] = df_inter2['k'].map(institutions_geo_mapz)
    #center of mass
    df_inter2['C_x'] = (df_inter2[['i_x','j_x','k_x']]).mean(axis=1)
    df_inter2['C_y'] = (df_inter2[['i_y','j_y','k_y']]).mean(axis=1)
    df_inter2['C_z'] = (df_inter2[['i_z','j_z','k_z']]).mean(axis=1)
    #Root Mean Square Radius, RMS radius
    df_inter2['i_dist'] = np.sqrt((df_inter2['i_x'] - df_inter2['C_x'])**2 + (df_inter2['i_y'] - df_inter2['C_y'])**2 + (df_inter2['i_z'] - df_inter2['C_z'])**2)
    df_inter2['j_dist'] = np.sqrt((df_inter2['j_x'] - df_inter2['C_x'])**2 + (df_inter2['j_y'] - df_inter2['C_y'])**2 + (df_inter2['j_z'] - df_inter2['C_z'])**2)
    df_inter2['k_dist'] = np.sqrt((df_inter2['k_x'] - df_inter2['C_x'])**2 + (df_inter2['k_y'] - df_inter2['C_y'])**2 + (df_inter2['k_z'] - df_inter2['C_z'])**2)
    df_inter2['dist_RMS'] =  np.sqrt((df_inter2[['i_dist','j_dist','k_dist']]**2).mean(axis=1))
    df_inter2 = df_inter2[['i','j','k','dist_mean','dist_max','dist_RMS']]
    df_inter2[['i', 'j', 'k']] = df_inter2[['i', 'j', 'k']].apply(lambda row: sorted(row), axis=1, result_type='expand')

    df_inter3['i_x'] = df_inter3['i'].map(institutions_geo_mapx)
    df_inter3['i_y'] = df_inter3['i'].map(institutions_geo_mapy)
    df_inter3['i_z'] = df_inter3['i'].map(institutions_geo_mapz)
    df_inter3['j_x'] = df_inter3['j'].map(institutions_geo_mapx)
    df_inter3['j_y'] = df_inter3['j'].map(institutions_geo_mapy)
    df_inter3['j_z'] = df_inter3['j'].map(institutions_geo_mapz)
    df_inter3['k_x'] = df_inter3['k'].map(institutions_geo_mapx)
    df_inter3['k_y'] = df_inter3['k'].map(institutions_geo_mapy)
    df_inter3['k_z'] = df_inter3['k'].map(institutions_geo_mapz)
    #center of mass
    df_inter3['C_x'] = (df_inter3[['i_x','j_x','k_x']]).mean(axis=1)
    df_inter3['C_y'] = (df_inter3[['i_y','j_y','k_y']]).mean(axis=1)
    df_inter3['C_z'] = (df_inter3[['i_z','j_z','k_z']]).mean(axis=1)
    #Root Mean Square Radius, RMS radius
    df_inter3['i_dist'] = np.sqrt((df_inter3['i_x'] - df_inter3['C_x'])**2 + (df_inter3['i_y'] - df_inter3['C_y'])**2 + (df_inter3['i_z'] - df_inter3['C_z'])**2)
    df_inter3['j_dist'] = np.sqrt((df_inter3['j_x'] - df_inter3['C_x'])**2 + (df_inter3['j_y'] - df_inter3['C_y'])**2 + (df_inter3['j_z'] - df_inter3['C_z'])**2)
    df_inter3['k_dist'] = np.sqrt((df_inter3['k_x'] - df_inter3['C_x'])**2 + (df_inter3['k_y'] - df_inter3['C_y'])**2 + (df_inter3['k_z'] - df_inter3['C_z'])**2)
    df_inter3['dist_RMS'] =  np.sqrt((df_inter3[['i_dist','j_dist','k_dist']]**2).mean(axis=1))
    df_inter3 = df_inter3[['i','j','k','dist_mean','dist_max','dist_RMS']]

    df_concat = pd.concat([df_intra,df_inter2,df_inter3])

    my_file = "inst_set.pickle"
    with open(os.path.join(path_3authors, my_file),"rb") as fp:
        inst_set = pickle.load(fp)
    my_file = "dfs_3authors.pickle"
    with open(os.path.join(path_3authors, my_file),"rb") as fp:
        [works_3authors,works,works_authors_aff] = pickle.load(fp) 

    inst_set_s_compl = inst_set - inst_set_s
    inst_set_s_compl = inst_set_s_compl.union(set(works_authors_aff.institution_id)-inst_set)
    works_outside = set(works_authors_aff[works_authors_aff.institution_id.isin(inst_set_s_compl)].work_id)    
    works = works[~works.work_id.isin(works_outside)]
    works_authors_aff = works_authors_aff[~works_authors_aff.work_id.isin(works_outside)]    
    works_authors_aff = works_authors_aff[['work_id','institution_id']].reset_index()
    works_authors_aff = works_authors_aff.sort_values(by=['publication_date_1','work_id','institution_id'])
    works_authors_aff = works_authors_aff.groupby(['publication_date_1','work_id']).institution_id.apply(list).to_frame()
    works_authors_aff['i'] = works_authors_aff['institution_id'].apply(lambda x: x[0])
    works_authors_aff['j'] = works_authors_aff['institution_id'].apply(lambda x: x[1])
    works_authors_aff['k'] = works_authors_aff['institution_id'].apply(lambda x: x[2])
    works_authors_aff['i'] = works_authors_aff['i'].astype(int)
    works_authors_aff['j'] = works_authors_aff['j'].astype(int)
    works_authors_aff['k'] = works_authors_aff['k'].astype(int)
    works_authors_aff = works_authors_aff[['i','j','k']].reset_index()
    works_authors_aff = works_authors_aff.groupby(['publication_date_1','i','j','k']).work_id.count().to_frame().reset_index().rename(columns={'work_id':'count'})
    #works_authors_aff = works_authors_aff[works_authors_aff.publication_date_1<'2023']

    works_authors_aff = works_authors_aff.set_index('publication_date_1')
    start_year_0 = 2000
    end_year_0 = 2009
    start_year_1 = 2010
    end_year_1 = 2014
    start_year_2 = 2015
    end_year_2 = 2019
    start_year_3 = 2020
    end_year_3 = 2021
    start_year_4 = 2022
    end_year_4 = 2023
    works_authors_aff.loc[str(start_year_0):str(end_year_0),'period'] = 0
    works_authors_aff.loc[str(start_year_1):str(end_year_1),'period'] = 1
    works_authors_aff.loc[str(start_year_2):str(end_year_2),'period'] = 2
    works_authors_aff.loc[str(start_year_3):str(end_year_3),'period'] = 3
    works_authors_aff.loc[str(start_year_4):str(end_year_4),'period'] = 4
    works_authors_aff['period'] = works_authors_aff['period'].astype(int)

    df = pd.DataFrame()
    for y in range(5):
        df_y = works_authors_aff[works_authors_aff.period == y]
        df_y = df_y.merge(df_concat,on=['i','j','k'],how='right')#[['i','j','k','dist','count']]
        df_y['count'] = df_y['count'].fillna(0)
        df_y = df_y.groupby(['i','j','k','dist_mean','dist_max','dist_RMS'])['count'].sum().to_frame().reset_index()
        df_y['prob'] = df_y['count']/df_y['count'].sum()
        df_y['period'] = y
        df = pd.concat([df,df_y])
    my_file = "triangles_df_tail"+str(n)+".csv"    
    df.to_csv(os.path.join(path_3authors, my_file),index=False)      

In [ ]:
power_law_threshold_triangles(50)

In [ ]:
n = 50
my_file = "triangles_df_tail"+str(n)+".csv" 
triangles_df = pd.read_csv(os.path.join(path_3authors, my_file))  
my_file = "degree_df_tail"+str(n)+".csv"    
degree_df = pd.read_csv(os.path.join(path_3authors, my_file))

In [ ]:
plots(50,path_3authors)

In [ ]:
def plots(n,my_path_):
    if n==0:
        my_file_add = ""
    else:
        my_file_add = "_tail"+str(n)    
    
    periods_list = [0,1,2,3,4]
    periods_labels = ['[2000,2009]','[2010,2014]','[2015,2019]','[2020,2021]','[2022,2023]']
    color_dict = {0: 'yellow', 1: 'magenta', 2: 'mediumseagreen', 3: 'cyan', 4: 'orange'}
    
    my_file = "triangles_df"+my_file_add+".csv"
    edges_df_tot = pd.read_csv(my_path_ / my_file) 
    my_file = "degree_df"+my_file_add+".csv"
    degree_df = pd.read_csv(my_path_ / my_file)  
    
    def mean_scatter_plot_multi(periods_list,periods_labels,color_dict,func,edges_df,degree_df,col,columnx,columny,labelx,labely,title,logx=False,logy=False,limity=False,geomspace=False):
        fig, ax = plt.subplots(figsize=(8,5))
        for y in periods_list[1:]:
            df1 = func(edges_df,degree_df,col,y)
            x1 = np.array(df1[columnx]) 
            y1 = np.array(df1[columny])  
            # Define the grid
            gridsize = 10**2
            if geomspace:
                xbins1 = np.geomspace(x1.min(), x1.max(), gridsize)
                #xbins2 = np.geomspace(x2.min(), x2.max(), gridsize)
            else:
                xbins1 = np.linspace(x1.min(), x1.max(), gridsize)
                #xbins2 = np.linspace(x2.min(), x2.max(), gridsize)
            # Calculate the mean values within each column
            mean_values1 = []
            for i in range(len(xbins1) - 1):
                mask = (x1 >= xbins1[i]) & (x1 < xbins1[i + 1])
                mean_y1 = np.mean(y1[mask])
                mean_values1.append(mean_y1) 
            ax.plot((xbins1[:-1] + xbins1[1:]) / 2, mean_values1, '.',color=color_dict[y],markersize=5, label=periods_labels[y])          
        ax.set_xlabel(labelx,size=20)
        ax.set_ylabel(labely,size=20)
        ax.set_title(title,size=30)
        if logx==True:
            ax.set_xscale('log')  
        if logy==True:
            ax.set_yscale('log')
        if limity==True:
            ax.set_ylim([0, 1])  
        ax.legend(bbox_to_anchor=(1.0, 0.9), prop={'size': 15}, markerscale=3)
        plt.show() 
    def scipy_fun_multi(periods_list,periods_labels,color_dict,func,edges_df,degree_df,col,x_max,labelx,labely,title):
        fig, ax = plt.subplots(figsize=(8,5))  
        for y in periods_list[1:]:
            x1,y1 = func(edges_df,degree_df,col,y)
            y1 = np.array(y1)
            popt1, pcov1 = scipy.optimize.curve_fit(linear, x1[x1<=x_max], y1[x1<=x_max])
            perr1 = np.sqrt(np.diag(pcov1))
            print(f'{periods_labels[y]}: gamma {popt1[0]} (perr {perr1[0]:.4f})') 
            ax.scatter(x1[x1<=x_max], y1[x1<=x_max],marker='.',color=color_dict[y],linewidths=0.01, label=periods_labels[y]) 
            ax.plot(x1[x1<=x_max], linear(x1[x1<=x_max], *popt1),color=color_dict[y],linewidth=2.0)   
        ax.set_xlabel(labelx,size=20)
        ax.set_ylabel(labely,size=20)
        ax.legend(bbox_to_anchor=(1.0, 0.9), prop={'size': 15}, markerscale=3)
        ax.set_title(title+" - linear",size=30)
        plt.show()
    def linear(x, b):
        return  b * x 
    def linear_fit(edges_df,degree_df,col,yy):
        df1 = fig_2B(edges_df,degree_df,col,yy)
        x1 = np.array(df1["s"]) 
        y1 = np.array(df1[col])  
        gridsize = 10**2
        xbins1 = np.linspace(x1.min(), x1.max(), gridsize)    
        mean_values1 = []
        for i in range(len(xbins1) - 1):
            mask = (x1 >= xbins1[i]) & (x1 < xbins1[i + 1])
            mean_y1 = np.mean(y1[mask])
            mean_values1.append(mean_y1)     
        x = (xbins1[:-1] + xbins1[1:]) / 2
        y = np.array(mean_values1)
        x = x[~np.isnan(y)]
        y = y[~np.isnan(y)]
        return x,y
    
    def fig_2B(edges_df,degree_df,col,y):
        edges_year_df = edges_df.query('period == @y')
        loops_w_count = edges_year_df.query('i==j and j==k')[['i',col]].rename(columns={'i':'institution_id'})
        degree_year_df = (degree_df[degree_df.period<=y][['institution_id','s']]).groupby(['institution_id']).s.sum().to_frame().reset_index()
        loops_w_count = loops_w_count.merge(degree_year_df,on='institution_id')
        return loops_w_count
    
    mean_scatter_plot_multi(periods_list,periods_labels,color_dict,fig_2B,edges_df_tot,degree_df,"weight","s","weight","s_i","w_ii",'Fig. 2B',False,False,False,False)
    scipy_fun_multi(periods_list,periods_labels,color_dict,linear_fit,edges_df_tot,degree_df,"weight",2*10**4,"s_i","w_ii","Fig. 2B")
    #mean_scatter_plot_multi(periods_list,periods_labels,color_dict,fig_2B,edges_df_tot,degree_df,"prob","s","prob","s_i","p_ii",'Fig. 2B',False,False,False,False)
    #scipy_fun_multi(periods_list,periods_labels,color_dict,linear_fit,edges_df_tot,degree_df,"prob",2*10**4,"s_i","p_ii","Fig. 2B")
    
    def fig_4B(df1,df2,col,y):
        df1 = df1.query('period == @y')[['i','j','k',col,'dist_mean']]
        df2 = df2.query('period == @y')[['institution_id','s']]
        df2_dict = df2.set_index('institution_id').to_dict()['s']
        df1['source_s'] = df1['source'].map(df2_dict)
        df1['target_s'] = df1['target'].map(df2_dict)
        df1['ratio'] = df1[col] / (df1['source_s']*df1['target_s'])**(1/2)
        df1 = df1[["dist", "ratio"]]
        #delate zero
        df1 = df1[df1.dist>0]
        return df1
    def scipy_fun_multi(periods_list,periods_labels,color_dict,func,edges_df,degree_df,col,x_min,labelx,labely,title):
        fig, ax = plt.subplots(figsize=(8,5))  
        for y in periods_list[1:]:
            x1,y1 = func(edges_df,degree_df,col,y)
            y1 = np.array(y1)
            popt1, pcov1 = scipy.optimize.curve_fit(power2, x1[x1>=x_min], y1[x1>=x_min])
            perr1 = np.sqrt(np.diag(pcov1))
            print(f'{periods_labels[y]}: alpha {(-1)*popt1[1]:.3f} (perr {perr1[1]:.4f}), beta {popt1[0]:.6f} (perr {perr1[0]:.4f})') 
            ax.scatter(x1[x1>=x_min], y1[x1>=x_min],marker='.',color=color_dict[y],linewidths=0.01, label=periods_labels[y]) 
            ax.plot(x1[x1>=x_min], power2(x1[x1>=x_min], *popt1),color=color_dict[y],linewidth=2.0)   
        ax.set_xscale('log')  
        ax.set_yscale('log')
        ax.set_xlabel(labelx,size=20)
        ax.set_ylabel(labely,size=20)
        ax.legend(bbox_to_anchor=(1.0, 0.9), prop={'size': 15}, markerscale=3)
        ax.set_title(title+" - power law",size=30)
        plt.show()
    def power2(x, b, c):
        return  b * x ** c 
    def powerlaw_fit(edges_df,degree_df,col,yy):
        df = fig_4B(edges_df,degree_df,col,yy)
        x = np.array(df["dist"]) 
        y = np.array(df["ratio"])
        gridsize = 10**2
        xbins = np.geomspace(x.min(), x.max(), gridsize)
        # Calculate the mean values within each column
        mean_values = []
        for i in range(len(xbins) - 1):
            mask = (x >= xbins[i]) & (x < xbins[i + 1])
            mean_y = np.mean(y[mask])
            mean_values.append(mean_y) 
        x = (xbins[:-1] + xbins[1:]) / 2
        y = np.array(mean_values)
        return x,y
    
    mean_scatter_plot_multi(periods_list,periods_labels,color_dict,fig_4B,edges_df_tot,degree_df,"weight","dist", "ratio","d_ij","w_ij / (s_i*s_j)^(1/2) ",'Fig. 4B',True,True,False,True)
    scipy_fun_multi(periods_list,periods_labels,color_dict,powerlaw_fit,edges_df_tot,degree_df,"weight",10**1,"d_ij","w_ij / (s_i*s_j)^(1/2) ","Fig. 4B - power law")
    scipy_fun_multi(periods_list,periods_labels,color_dict,powerlaw_fit,edges_df_tot,degree_df,"weight",10**2,"d_ij","w_ij / (s_i*s_j)^(1/2) ","Fig. 4B - power law")
    #mean_scatter_plot_multi(periods_list,periods_labels,color_dict,fig_4B,edges_df_tot,degree_df,"prob","dist", "ratio","d_ij","p_ij / (s_i*s_j)^(1/2) ",'Fig. 4B',True,True,False,True)
    #scipy_fun_multi(periods_list,periods_labels,color_dict,powerlaw_fit,edges_df_tot,degree_df,"prob",10**1,"d_ij","p_ij / (s_i*s_j)^(1/2) ","Fig. 4B - power law")
    #scipy_fun_multi(periods_list,periods_labels,color_dict,powerlaw_fit,edges_df_tot,degree_df,"prob",10**2,"d_ij","p_ij / (s_i*s_j)^(1/2) ","Fig. 4B - power law")    

## Edges - all works 

In [ ]:
path_allauthors = Path('./Model_allworks')
if not os.path.exists(path_allauthors):
    os.makedirs(path_allauthors)

### Tables

In [ ]:
works = read_parquet(basepath / 'works')
works_authors_aff = read_parquet(basepath / 'works_authors_aff')

works = works.loc['2000-01-01':'2023-12-01'] 
works = works[works.num_authors>1]

works_allauthors = set(works.work_id)
print(f'{len(works_allauthors)} ({(len(works_allauthors)/len(works))*100:.2f}%) works all authors')

works = works[works.work_id.isin(works_allauthors)]
works_authors_aff = works_authors_aff[works_authors_aff.work_id.isin(works_allauthors)]

my_file = "dfs_allauthors.pickle"
pickle.dump([works_allauthors,works,works_authors_aff], open(os.path.join(path_allauthors, my_file), 'wb')) 

In [ ]:
#INITIALIZATION #preferential attachment  #TW: 2000-2009
my_file = "dfs_allauthors.pickle"
with open(os.path.join(path_allauthors, my_file),"rb") as fp:
    [works_allauthors,works,works_authors_aff] = pickle.load(fp)  
works = works.loc['2000-01-01':'2023-12-01'] 
works_authors_aff = works_authors_aff.loc['2000-01-01':'2023-12-01']

N_dict = works_authors_aff.groupby('publication_date_1').work_id.nunique().to_frame().to_dict()['work_id']
months_list = list(N_dict.keys())
months_list.sort()
months_list = [i.strftime('%Y-%m-%d') for i in months_list]

start_index = 0
end_index = 120 #180
start_month = months_list[start_index]#.strftime('%Y-%m-%d')
end_month = months_list[end_index-1]#.strftime('%Y-%m-%d')

df_TW = works_authors_aff.loc[start_month:end_month] #df_TW = works_authors_aff.loc[months_list[:120]]
inst_set = set(df_TW.institution_id)
I = len(inst_set)
print(f'TW from {start_month} to {end_month} : {I} institutions')

In [ ]:
#initial strenghts    #consider only institutions in sample in TW
#count number (unique) institutions per paper
df_TW = df_TW.drop_duplicates(['work_id','institution_id'])
df_TW['num_affs'] = df_TW.groupby('work_id')['institution_id'].transform('size')
print(f'{min(df_TW.num_affs)}-{max(df_TW.num_affs)} min-max number (unique) affiliations per work')
df_TW['weight'] = 2 / ( df_TW['num_affs']*(df_TW['num_affs']-1) ) 
df_TW.loc[df_TW.num_affs==1,'weight'] = 1 #one affiliation
df_TW_noloops = df_TW[df_TW.num_affs>1]
df_TW_loops = df_TW[df_TW.num_affs==1]
df_TW_loops['institution_id2'] = df_TW_loops['institution_id']
df_TW_loops['weight'] = df_TW_loops[['institution_id','institution_id2','weight']].groupby(['institution_id']).weight.transform('sum')
df_TW_loops = df_TW_loops.drop_duplicates('institution_id')
I_graph = make_institution_graph(df_TW_noloops)
I_graph.add_weighted_edges_from([tuple(r) for r in df_TW_loops[['institution_id','institution_id2','weight']].to_numpy()])
df_ = pd.DataFrame.from_dict(dict(I_graph.degree(weight='weight')),orient='index').reset_index().rename(columns={'index':'institution_id',0:'strength'})
df_['institution_id'] = df_['institution_id'].astype(int)
df = df_.sort_values(by='strength',ascending=False)
inst_set = set(df.institution_id)
I = len(inst_set)
print(f'TW from {start_month} to {end_month} : {I} institutions')

my_file = "df_strengths0.csv"     
df.to_csv(os.path.join(path_allauthors, my_file),index=False)

my_file = "inst_set.pickle"
pickle.dump(inst_set, open(os.path.join(path_allauthors, my_file), 'wb'))

In [ ]:
I_dist = read_parquet(basepath / 'I_dist_threshold')
I_dist = I_dist[['source','target','dist']].reset_index(drop=True)
I_dist = I_dist.query('source.isin(@inst_set) & target.isin(@inst_set)').reset_index()
my_file = "I_dist_model.csv" 
I_dist.to_csv(os.path.join(path_allauthors, my_file),index=False)

In [ ]:
work_authors_edges_df_dist = read_parquet(Path('./TeamDistance') / 'work_authors_edges_df_dist')
work_authors_edges_df_dist = works[['work_id']].reset_index().merge(work_authors_edges_df_dist,on='work_id')
work_authors_edges_df_dist = work_authors_edges_df_dist[work_authors_edges_df_dist.work_id.isin(works_allauthors)]
work_edges_dist_mean = work_authors_edges_df_dist.groupby('work_id').dist.mean().to_frame().reset_index()
work_edges_dist_mean = work_edges_dist_mean.merge(works[['work_id']].reset_index(),on='work_id')
work_edges_dist_mean_monthly = work_edges_dist_mean.groupby('publication_date_1').dist.mean().to_frame().reset_index()
my_file = "work_edges_dist_mean.csv"    
work_edges_dist_mean.to_csv(os.path.join(path_allauthors, my_file),index=False)

In [ ]:
my_file = "work_edges_dist_mean_monthly.csv"     
work_edges_dist_mean_monthly.to_csv(os.path.join(path_allauthors, my_file),index=False) 

In [ ]:
df_data1 = df_data1_(work_authors_edges_df_dist)
df_data1['publication_date_1'] = df_data1['publication_date_1'].apply(pd.to_datetime)